The Omega Manufacturing Company has discontinued the production of a certain unprofitable product line. This act created considerable excess production capacity. Management is considering devoting this excess capacity to one or more of three products; call them products $1$, $2$, and $3$. The available capacity on the machines that might limit output is summarized in the following table:

| Machine Type | Available Time (Machine hours per week) |
| :--- | :---: |
| Miling machine | 500 |
| Lathe | 350 |
| Grinder | 150 |

The number of machine hours required for each unit of the respective products is

| Machine Type | Product 1 | Product 2 | Product 3 |
| :--- | :---: | :---: | :---: |
| Milling machine | 9 | 3 | 5 |
| Lathe | 5 | 4 | 0 |
| Grinder | 3 | 0 | 2 |


The sales department indicates that the sales potential for products $1$ and $2$ exceeds the maximum production rate and that the sales potential for product $3$ is $20$ units per week. The unit profit would be $\$50$, $\$20$, and $\$25$, respectively, on products $1$, $2$, and $3$. The objective is to determine how much of each product Omega should produce to maximize profit.


In [1]:
from pyomo.environ import *

In [2]:
model = AbstractModel()

In [3]:
model.products = Set()
model.machines = Set()

In [4]:
model.available_time = Param(model.machines)
model.machine_hour_per_product = Param(model.machines, model.products)
model.unit_profit = Param(model.products)

In [5]:
model.quotas = Var(model.products, within=NonNegativeIntegers)

In [6]:
def total_profit(model):
    return summation(model.unit_profit, model.quotas)


model.profit = Objective(rule=total_profit, sense=maximize)

In [7]:
def maximum_machine_hours(model, machine):
    return sum(model.machine_hour_per_product[machine, product] * model.quotas[product] for product in model.products) <= model.available_time[machine]

model.machines_constraint = Constraint(model.machines, rule=maximum_machine_hours)    

In [8]:
def sales_potential(model, product):
    if product == 3:
        return model.quotas[product] <= 20
    else:
        return Constraint.Feasible

model.sales_potential_constraint = Constraint(model.products, rule=sales_potential)

In [9]:
data = { None: { 'products': { None: (1, 2, 3 ) },
                 'machines': { None: ('Milling machine', 'Lathe', 'Grinder')} ,
                 'available_time': { 'Milling machine': 500,
                                     'Lathe': 350,
                                     'Grinder': 150
                                   },
                 'machine_hour_per_product': { ('Milling machine', 1): 9,
                                               ('Milling machine', 2): 3,
                                               ('Milling machine', 3): 5,
                                               ('Lathe', 1): 5,
                                               ('Lathe', 2): 4,
                                               ('Lathe', 3): 0,
                                               ('Grinder', 1): 3,
                                               ('Grinder', 2): 0,
                                               ('Grinder', 3): 2
                                             },
                 'unit_profit': { 1: 50,
                                  2: 20,
                                  3: 25
                                }
               } 
       }

In [10]:
instance = model.create_instance(data)
instance.pprint()

3 Set Declarations
    machine_hour_per_product_index : Dim=0, Dimen=2, Size=9, Domain=None, Ordered=False, Bounds=None
        Virtual
    machines : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=None
        ['Grinder', 'Lathe', 'Milling machine']
    products : Dim=0, Dimen=1, Size=3, Domain=None, Ordered=False, Bounds=(1, 3)
        [1, 2, 3]

3 Param Declarations
    available_time : Size=3, Index=machines, Domain=Any, Default=None, Mutable=False
        Key             : Value
                Grinder :   150
                  Lathe :   350
        Milling machine :   500
    machine_hour_per_product : Size=9, Index=machine_hour_per_product_index, Domain=Any, Default=None, Mutable=False
        Key                    : Value
                ('Grinder', 1) :     3
                ('Grinder', 2) :     0
                ('Grinder', 3) :     2
                  ('Lathe', 1) :     5
                  ('Lathe', 2) :     4
                  ('Lathe', 3) :     0
        ('Mil

In [11]:
opt = SolverFactory('glpk')

In [12]:
results = opt.solve(instance)
instance.solutions.store_to(results)
print(results)


Problem: 
- Name: unknown
  Lower bound: 2900.0
  Upper bound: 2900.0
  Number of objectives: 1
  Number of constraints: 5
  Number of variables: 4
  Number of nonzeros: 9
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.01625370979309082
Solution: 
- number of solutions: 1
  number of solutions displayed: 1
- Gap: 0.0
  Status: optimal
  Message: None
  Objective:
    profit:
      Value: 2900
  Variable:
    quotas[1]:
      Value: 26
    quotas[2]:
      Value: 55
    quotas[3]:
      Value: 20
  Constraint: No values

